# Methods and Plan
### *Muhan Yang - Group 11*

In [44]:
library(tidyverse)
library(GGally)
library(mltools)
library(leaps)

In [45]:
# github link to csv dataset
link <- "https://raw.githubusercontent.com/splashhhhhh/stat301/main/repositories.csv"

# read data
data <- read.csv(link)

# see first few lines of the dataset
# head(data)

## Data Description (Copied from Data & Question)

The dataset we are using is the *Most Popular Github Repositories (Projects)* dataset from Kaggle (URL: https://www.kaggle.com/datasets/donbarbos/github-repos/data), which contains a list of top GitHub project repositories by the number of stars. The data is collected using GitHub search API and the query function to obtain projects with over 167 stars.

The repositories dataset consists of 215,029 project repositories (i.e., rows/observations) and 24 columns (i.e., variables) in total. Of the 24 columns, there are five columns containing the integer type of data of repositories: Size, Stars, Forks, Issues, and Watchers, and the data type of the rest 19 variables are characters. Of the 19 columns, there are nine binary variables that contain data coded as "True" or "False" - Has.Issues, Has.Projects, Has.Downloads, Has.Wiki, Has.Pages, Has.Discussions, Is.Fork, Is.Archived, and Is.Template. There are also two columns, consisting temporal data as character data type, within the rest ten variables: Created.At and Updated.At. 

- `Name`           : chr - name of the repository
- `Description`    : chr - brief description of the repository
- `URL`            : chr - github url
- `Created.At`     : chr - date and time of the creation
- `Updated.At`     : chr - date and time of the latest update
- `Homepage`       : chr - home page url
- `Size`           : int - size of the repository in bytes
- `Stars`          : int - number of stars/likes the repository received
- `Forks`          : int - number of times other users forked the repository
- `Issues`         : int - number of open issues
- `Watchers`       : int - number of watchers
- `Language`       : chr - programming language used
- `License`        : chr - software license information using a license identifier
- `Topics`         : chr - a list of topics/tags
- `Has.Issues`     : chr - whether the repository has an issue tracker or not
- `Has.Projects`   : chr - whether the repository uses GitHub Projects to manage and organize tasks and work items
- `Has.Downloads`  : chr - whether the repository offers downloadable files to users
- `Has.Wiki`       : chr - whether the repository has a related wiki page
- `Has.Pages`      : chr - whether the repository has GitHub Pages enabled
- `Has.Discussions`: chr - whether the repository has GitHub Discussions enabled
- `Is.Fork`        : chr - whether the repository is a fork of another repository
- `Is.Archived`    : chr - whether the repository is archived
- `Is.Template`    : chr - whether the repository is set up as a template
- `Default.Branch` : chr - the name of the default branch

Above is a list of bulletpoints of the name, the type of each variable, and its descriptions.

In [46]:
# very brief wrangling to exclude unrelated columns
data <- data %>%
    select(-Description, -Language, -Created.At, -Updated.At, -URL, -Homepage, -Topics, -License)
# head(data)

In [47]:
# check NAs, 0s, and empty values
colSums(data==0)
colSums(data=="")

Name            Size           Stars           Forks          Issues 
              0             151               0             216           23647 
       Watchers      Has.Issues    Has.Projects   Has.Downloads        Has.Wiki 
              0               0               0               0               0 
      Has.Pages Has.Discussions         Is.Fork     Is.Archived     Is.Template 
              0               0               0               0               0 
 Default.Branch 
              0

Name            Size           Stars           Forks          Issues 
              2               0               0               0               0 
       Watchers      Has.Issues    Has.Projects   Has.Downloads        Has.Wiki 
              0               0               0               0               0 
      Has.Pages Has.Discussions         Is.Fork     Is.Archived     Is.Template 
              0               0               0               0               0 
 Default.Branch 
              0

In [48]:
# Filter out 0s and empty values
data <- data %>%
    filter(Size != 0, Forks != 0, Issues != 0) %>%
    filter(Name != "")

In [49]:
# table(data$Default.Branch)

According to the frequency table of default branch, most of its data is in main and master. Therefore, we will filter out the rest.

In [50]:
data <- data %>%
    filter(Default.Branch == "main" | Default.Branch == "master")
# head(data)

Since the dataset contains huge number of observations, we are only going to use 3,000 observations randomly drawn from the entire data.

In [51]:
# random selection of 3,000 observations
data_s <- sample_n(data, 3000, replace = FALSE)

In [52]:
# head(data_s)

## Question

#### What characteristic(s) of the GitHub repository can help to predict the number of stars/likes the repository received?

This question is refined from Data & Question submission.

In the question above, the response variable is the number of stars/likes the repository received, which is the information from the `Stars` column. As of predictors, this question is an exploratory one and is aiming to find a best predictive model for `Stars`, using the knowledge from model selection and evaluation.

The question is mostly proposed for making predictions on the number of stars the top repositories received based on the data. However, if we found any coefficients significant, we might be able to extend the results to the overall population of the entire GitHub repositories. However, by doing this, we should always bear in mind that our data only sampled from those repositories with top numbers of stars (over 167 stars). Therefore, our results might not be able to generate to the entire population and thus the inference might not work well for the all repositories on GitHub due to restriction of range.

### Get Descriptive Statistics Summary for Numerical Data

In [53]:
# select numerical variables out
data_num <- data_s %>%
    select(Size, Stars, Forks, Issues, Watchers)

In [54]:
# convert the dataset into a long format
data_long <- data_num %>%
    gather(factor_key=TRUE)
# head(data_long)

In [55]:
# obtain descriptive stats summary table
data_stats <- data_long %>% group_by(key)%>%
 summarise(mean= mean(value), sd= sd(value), max = max(value),min = min(value))
# data_stats

In [56]:
# select all categorical or binary data, except name
data_cat <- data_s %>%
    select(-Name, -Size, -Stars, -Forks, -Issues, -Watchers)

#### Get frequency table for binary data

In [57]:
table(data_cat$Has.Projects)


False  True 
  386  2614 

In [58]:
table(data_cat$Has.Issues)


False  True 
   81  2919 

In [59]:
table(data_cat$Has.Downloads)


False  True 
   24  2976 

In [60]:
table(data_cat$Has.Wiki)


False  True 
  591  2409 

In [61]:
table(data_cat$Has.Pages)


False  True 
 2511   489 

In [62]:
table(data_cat$Has.Discussions)


False  True 
 2641   359 

In [63]:
table(data_cat$Is.Fork)


False 
 3000 

In [64]:
table(data_cat$Is.Archived)


False  True 
 2810   190 

In [65]:
table(data_cat$Is.Template)


False  True 
 2985    15 

In [66]:
table(data_cat$Default.Branch)


  main master 
   583   2417 

Since `Has.Issues`, `Has.Downloads`, `Is.Fork`, and `Is.Template` are close to only having one value (i.e., the frequency difference between the binary outcome is too small), we will not look at the data of these columns as they might not be reliable in terms of making inferences or prediction because they are not balanced.

In [67]:
data_s <- data_s %>%
    select(-Has.Issues, -Has.Downloads, -Is.Fork, -Is.Template)
# head(data_s)

### Check the Correlation and Distribution of Numerical Variables

In [68]:
options(repr.plot.width = 8, repr.plot.height = 6)

data_pair_plots <- data_num %>%
  ggpairs(progress = FALSE) +
  theme(
    text = element_text(size = 15),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold")
  )

Since watchers and stars have the perfect correlation of 1, we will only use one of them - `Stars`. From the correlation above, we can see that each numerical variable has a lot of outliers, which makes most points stick at the bottom left corner of each scatterplot.

In [69]:
data_s <- data_s %>%
    select(-Watchers)
data_num <- data_num %>%
    select(-Watchers)

In [70]:
# eliminate outliers from numerical variables
outliers_size <- boxplot(data_s$Size, plot=FALSE)$out
data_s <- data_s[-which(data_s$Size %in% outliers_size),]

outliers_stars <- boxplot(data_s$Stars, plot=FALSE)$out
data_s <- data_s[-which(data_s$Stars %in% outliers_stars),]

outliers_forks <- boxplot(data_s$Forks, plot=FALSE)$out
data_s <- data_s[-which(data_s$Forks %in% outliers_forks),]

outliers_issues <- boxplot(data_s$Issues, plot=FALSE)$out
data_s <- data_s[-which(data_s$Issues %in% outliers_issues),]

In [71]:
# generate numerical variable sub-dataset
data_num2 <- data_s %>%
    select(Size, Stars, Forks, Issues)

In [72]:
# relook at the correlation between numerical variables
data_pair_plots <- data_num2 %>%
  ggpairs(progress = FALSE) +
  theme(
    text = element_text(size = 15),
    plot.title = element_text(face = "bold"),
    axis.title = element_text(face = "bold")
  )

In [73]:
# create a melted correlation matrix
corr_matrix_data <- data_num2 %>%
  cor() %>%
  as.data.frame() %>%
  rownames_to_column("var1") %>%
  pivot_longer(-var1, names_to = "var2", values_to = "corr")

In [74]:
# create a heatmap with correlation coefficients after dropping outliers
plot_corr_matrix_data <- corr_matrix_data %>%
  ggplot(aes(x = var1, y = var2)) +
  geom_tile(aes(fill = corr), color = "white") +
  scale_fill_distiller("Correlation Coefficient \n",
    palette =  "YlOrRd",
    direction = 1, limits = c(-1, 1)
  ) +
  labs(x = "first input variable", y = "second input variable") +
  theme_minimal() +
  theme(
    axis.text.x = element_text(
      angle = 45, vjust = 1,
      size = 18, hjust = 1
    ),
    axis.text.y = element_text(
      vjust = 1,
      size = 18, hjust = 1
    ),
    legend.title = element_text(size = 18, face = "bold"),
    legend.text = element_text(size = 18),
    legend.key.size = unit(2, "cm")
  ) +
  coord_fixed() +
  geom_text(aes(x = var1, y = var2, label = round(corr, 2)), color = "black", size = 6)


From the heatmap above, we can see that the most significant numeric predictor of `Stars` would be `Forks`, and the relationship of `Size` and the other three is the least correlated among these numerical variables.

In [75]:
# recode categorical variables
data_s$Has.Projects<-ifelse(data_s$Has.Projects=="True",1,0)
data_s$Has.Wiki<-ifelse(data_s$Has.Wiki=="True",1,0)
data_s$Has.Pages<-ifelse(data_s$Has.Pages=="True",1,0)
data_s$Has.Discussions<-ifelse(data_s$Has.Discussions=="True",1,0)
data_s$Is.Archived<-ifelse(data_s$Is.Archived=="True",1,0)
data_s <- data_s %>%
    mutate(Default.Branch = recode(Default.Branch, "main" = 0, "master" = 1))

In [76]:
# do last bit of wrangling
data_s <- data_s %>%
    select(-Name)

## Methods and Plan

Since we are doing a prediction question - What characteristic(s) of the GitHub repository can help to predict the number of stars/likes the repository received, we are going to first split the sample data into training set and testing set using a 70-30 basis, use training set to determine a well-trained model using Linear Regression, and test our model using the testing set.

Using the forward stepwise selection, we can use the BIC (Bayesian Information Criterion) of each model to select the model, since we want the model to be predictive rather than generative. We can also plot the Cp plot of the model out and select the minimum Cp model. Also, BIC can be used to approximate the test MSE, without looking at the test data.

One potential limitation of the methods is that our test would depend on the random split of the data, and different split might lead to different selection of variables to the best predictive model. Also, since we are using the random sample from a large dataset, it might not be accurate enough to capture the full population.

## Implementation of a Proposed Model

In [77]:
data_s$ID <- rownames(data_s)
training_data <- sample_n(data_s, size = nrow(data_s) * 0.70,
  replace = FALSE
)

testing_data <- anti_join(data_s,
  training_data,
  by = "ID"
)

training_data <- training_data %>% select(-"ID")
testing_data <- testing_data %>% select(-"ID")


In [78]:
# build an additive predictive model
data_full_OLS <- lm(Stars ~ .,
  training_data
)
# data_full_OLS

In [79]:
# obtain the (out-of-sample) predicted values
data_test_pred_full_OLS <- predict(data_full_OLS, newdata = testing_data)
head(data_test_pred_full_OLS)

1        2        3        4        5        6 
397.6711 445.4221 425.6250 394.4300 342.1784 301.0666

In [80]:
# compute the Root Mean Squared Error (RMSE) using data from the test set 
data_RMSE_models <- tibble(
  Model = "OLS Full Regression",
  RMSE = rmse(
    data_test_pred_full_OLS,
    testing_data$Stars
  )
)
# data_RMSE_models

In [81]:
# select a reduced LR using the forward selection algorithm from training set
data_forward_sel <- regsubsets(
  x = Stars ~., nvmax = 9,
  data = training_data,
  method = "forward"
)
# data_forward_sel

data_fwd_summary <- summary(data_forward_sel)

data_fwd_summary <- tibble(
   n_input_variables = 1:9,
   RSS = data_fwd_summary$rss,
   BIC = data_fwd_summary$bic,
   Cp = data_fwd_summary$cp
)


In [82]:
# data_fwd_summary
# summary(data_forward_sel)

In [83]:
# Identify the size of the model that minimizes Cp
cp_min = which.min(data_fwd_summary$Cp)

# Find the name of the variables for the best model
selected_var <- names(coef(data_forward_sel, cp_min))[-1]

# Reduce dataset to only include the selected predictors
training_subset <- training_data %>% select(all_of(selected_var),Stars)

# Train the predictive model
data_red_OLS <- lm(Stars ~ .,
  data = training_subset
)

# summary(data_red_OLS)

In [84]:
# use the trained model to predict the responses of the testing set
data_test_pred_red_OLS <- predict(data_red_OLS, newdata = testing_data)

In [85]:
# compute the RMSE of predicted stars in testing set
data_RMSE_models <- rbind(
  data_RMSE_models,
  tibble(
    Model = "OLS Reduced Regression",
    RMSE = rmse(data_test_pred_red_OLS, testing_data$Stars)
    )
  )
data_RMSE_models

Model,RMSE
<chr>,<dbl>
OLS Full Regression,231.3900
OLS Reduced Regression,231.9106


It turns out that the full regression model has a better out-of-sample prediction performance compared to our reduced ones. However, note that this is only a one-time estimate of the true test RMSE based on a random split of the data. If we split the data in a different way, we might be very likely to obtain a different result, given that the RMSE value difference between the full regression and the reduced regression is trivial.

Note: Since the teaching team said on Piazza that we should only keep one table/visualization in our entire assignment 4, I kept the very last result table and commented out other output codes.